In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from alpaca_trade_api import REST, TimeFrame
import datetime
import matplotlib.pyplot as plt
import logging

# Alpaca API credentials
API_KEY = 'YOUR_ALPACA_API_KEY'
SECRET_KEY = 'YOUR_ALPACA_SECRET_KEY'
BASE_URL = 'https://paper-api.alpaca.markets'

# Alpaca API client
alpaca = REST(API_KEY, SECRET_KEY, base_url=BASE_URL)

# Logging configuration
logging.basicConfig(filename='trading_log.log', level=logging.INFO, format='%(asctime)s - %(message)s')

def fetch_historical_data(symbol, start_date, end_date):
    return yf.download(symbol, start=start_date, end=end_date)

def fetch_real_time_data(symbol, start, end):
    return alpaca.get_barset(symbol, TimeFrame.Minute, start=start, end=end).df[symbol]

def calculate_moving_averages(prices, short_window=20, long_window=50):
    short_ma = prices.rolling(window=short_window).mean()
    long_ma = prices.rolling(window=long_window).mean()
    return short_ma, long_ma

def generate_signals(prices, short_ma, long_ma):
    signals = pd.DataFrame(index=prices.index)
    signals['price'] = prices
    signals['short_ma'] = short_ma
    signals['long_ma'] = long_ma
    signals['signal'] = 0
    signals['signal'][short_ma > long_ma] = 1
    signals['signal'][short_ma <= long_ma] = 0
    signals['positions'] = signals['signal'].diff()
    return signals

def execute_trade(symbol, action, quantity):
    if action == 'buy':
        order = alpaca.submit_order(symbol=symbol, qty=quantity, side='buy', type='market', time_in_force='gtc')
        logging.info(f'BUY: {symbol} - Quantity: {quantity}')
    elif action == 'sell':
        order = alpaca.submit_order(symbol=symbol, qty=quantity, side='sell', type='market', time_in_force='gtc')
        logging.info(f'SELL: {symbol} - Quantity: {quantity}')

def plot_signals(signals):
    plt.figure(figsize=(14, 7))
    plt.plot(signals['price'], label='Price')
    plt.plot(signals['short_ma'], label='20-day MA')
    plt.plot(signals['long_ma'], label='50-day MA')
    plt.plot(signals[signals['positions'] == 1].index, signals['short_ma'][signals['positions'] == 1], '^', markersize=10, color='g', lw=0, label='Buy Signal')
    plt.plot(signals[signals['positions'] == -1].index, signals['short_ma'][signals['positions'] == -1], 'v', markersize=10, color='r', lw=0, label='Sell Signal')
    plt.legend()
    plt.show()

def main():
    symbol = 'AAPL'
    start_date = '2023-01-01'
    end_date = '2023-01-31'
    initial_capital = 100000
    position_size = 10  # Number of shares to buy/sell

    # Fetch historical data for initial backtest
    historical_data = fetch_historical_data(symbol, start_date, end_date)
    prices = historical_data['Close']

    short_ma, long_ma = calculate_moving_averages(prices)
    signals = generate_signals(prices, short_ma, long_ma)
    plot_signals(signals)

    # Simulate real-time trading
    start_time = datetime.datetime.now()
    end_time = start_time + datetime.timedelta(days=1)  # Simulate for one day

    while datetime.datetime.now() < end_time:
        now = datetime.datetime.now()
        start = (now - datetime.timedelta(minutes=30)).isoformat()
        end = now.isoformat()
        real_time_data = fetch_real_time_data(symbol, start, end)
        prices = real_time_data['close']

        short_ma, long_ma = calculate_moving_averages(prices)
        signals = generate_signals(prices, short_ma, long_ma)

        if signals['positions'].iloc[-1] == 1:
            execute_trade(symbol, 'buy', position_size)
        elif signals['positions'].iloc[-1] == -1:
            execute_trade(symbol, 'sell', position_size)

        time.sleep(60)  # Sleep for 1 minute

if __name__ == "__main__":
    main()
